In [ ]:
import requests
import numpy as np
import tensorflow as tf
import msgpack
import json
import zlib
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.int64)
x_train = x_train.reshape(-1, 28*28)


In [ ]:

r = requests.get('http://localhost:3030/')

print(r)
print(r.text)

In [ ]:
sample_bytes = zlib.compress(msgpack.packb([float(f) for f in x_train[0]]))

r = requests.get('http://localhost:3030/knn', params = {"k": 5}, headers = {"Content-Type": "gzip"}, data=sample_bytes)

print(r)
print(json.loads(r.text))